In [2]:
# spark.stop()

In [3]:
import os
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, BooleanType

In [4]:
SPARK_MASTER = "spark://spark-master:7077"

spark = (
    SparkSession
    .builder
    # .master(SPARK_MASTER)
    .appName("process_type_mismatch")
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.14.jar") # GCS Connector
    .getOrCreate()
)

# Google Cloud Service Account Credentials
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))

spark

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 22:06:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
from google.cloud import storage

def get_year(blob):
    return blob.name.split("/")[3]

def get_filename(blob):
    return blob.name.split('/')[-1].split('.')[0]

def scan_years(blobs): 
    return list({get_year(blob) for blob in blobs})

def process_parquet(bucket, dir, year):
    blobs = list(bucket.list_blobs(prefix=f"{dir}/{year}"))
    for blob in blobs:
        source_blob = f"gs://{bucket.name}/{blob.name}"
        destination_blob = f"gs://{bucket.name}/processed/pq/{dir.split('/')[-1]}/{year}/{get_filename(blob)}"

        print(f"Reading file {source_blob}")
        # Read parquet
        df = spark.read.parquet(source_blob)

        # Castnig to String
        for col in df.columns:
            df = df.withColumn(col, F.col(col).cast(StringType()))

        # Write to Destination
        df.repartition(4).write.mode("overwrite").parquet(destination_blob) 
        print(f"Saved to {destination_blob}")

client = storage.Client()
bucket = client.get_bucket("zoomcamp-454219-ade-pipeline")
dirs = ["data/pq/patient", "data/pq/drug", "data/pq/reaction"]

# Iterate through each source directory and each year over n parquet files
for dir in dirs:
    # years = scan_years(bucket.list_blobs(prefix=dir))
    years = ['2004']
    for year in years:
        process_parquet(bucket, dir, year)
    break

Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-1-of-20.parquet


Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-1-of-20
Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-10-of-20.parquet


Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-10-of-20
Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-11-of-20.parquet


Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-11-of-20
Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-12-of-20.parquet


Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-12-of-20
Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-13-of-20.parquet


...

Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-5-of-20
Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-6-of-20.parquet


Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-6-of-20
Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-7-of-20.parquet


Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-7-of-20
Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-8-of-20.parquet


Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-8-of-20
Reading file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-9-of-20.parquet


Saved to gs://zoomcamp-454219-ade-pipeline/processed/pq/patient/2004/drug-event-part-9-of-20
